In [36]:
import re

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import copy


options = Options()
options.headless = True

browser = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)

#browser.get('https://library.dongduk.ac.kr/#/search/ex?isbn=1%7Cl%7Ca%7C9791165075064&branch=1&material-type=1')
# 되는 케이스

browser.get('https://library.dongduk.ac.kr/#/search/ex?isbn=1%7Cl%7Ca%7C9788931004441&branch=1&material-type=1')
#9788931004441 새로운 케이스


#9788953934283 새로운 케이스
#browser.get('https://library.daejin.ac.kr/data_data_list.mir?search_keyword_type1=isbn&search_keyword1=9788953934283&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=')
# 되는 케이스

#browser.get('https://discover.duksung.ac.kr/#/search/ex?isbn=1%7Ck%7Ca%7C9788934985099&material-type=1')
#안되는 케이스

In [38]:
linkXpath = '/html/body/div[1]/div[3]/div[2]/div[2]/div[2]/div[3]/div[1]/div[1]/div[2]/div[4]/div/div[3]/ul/li[1]/a[1]/span'
browserCheckXpath = '/html/body/div[1]/div[3]/div[2]/div[2]/div[1]/div[1]/h2'
#소장검색 이라는 단어

try:
    WebDriverWait(browser, 120).until(EC.presence_of_element_located((By.XPATH, browserCheckXpath)))
    linkElement = WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, linkXpath)))
    #linkElement = browser.find_element_by_xpath(linkXpath)
    # 오류가능성.
except :
    print('예외가 발생! 찾는 책이 없음')
    linkElement = -1

In [39]:
linkElement.text

'잃어버린 지평선'

In [40]:
if linkElement == -1 :
    print('-1...')
else : 
    print('success')

success


In [41]:
linkElement.click()

In [42]:
browser.current_url

'https://library.dongduk.ac.kr/#/search/detail/40078453'

In [43]:
ISBNLinkElementXpath = '//*[@id="btn-biblio-more-open"]'
ISBNLinkElement = WebDriverWait(browser, 120).until(EC.presence_of_element_located((By.XPATH, ISBNLinkElementXpath)))
#ISBNLinkElement = browser.find_element_by_xpath(ISBNLinkElementXpath)
ISBNLinkElement.click()

In [44]:
html = browser.page_source
bsObject=BeautifulSoup(html, 'html.parser')
strISBNIncluded = str(bsObject.select('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div.ikc-content > div.ikc-main > div:nth-child(3) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail'))

In [45]:
strISBNIncluded

'[<div class="ikc-biblio-detail"><div class="ikc-book-thumb"><div ng-click="showBookThumb=!showBookThumb"><ik-book-thumb bid="biblioId" default="none" isbn="targetISBN" thumbnail="biblioSummary.thumbnail" title="잃어버린 지평선"><div class="ikc-bookcover" ng-show="thumbShowFlag">\n<img alt="잃어버린 지평선" ng-src="https://image.aladin.co.kr/product/46/22/cover/8931004443_1.jpg" onerror="this.src=\'/assets/images/dd/content/default-item-img.jpg\'" src="https://image.aladin.co.kr/product/46/22/cover/8931004443_1.jpg"/>\n</div></ik-book-thumb></div><div class="ikc-dylink"><a class="button ng-hide" ng-href="" ng-show="ePassPreviewLink.length &gt; 0" target="_blank" title="새창">Core Collection\xa0<i class="k-icon k-i-search"></i></a> <a class="button" href="http://www.aladin.co.kr/shop/book/wletslookViewer.aspx?ISBN=8931004443" ng-href="http://www.aladin.co.kr/shop/book/wletslookViewer.aspx?ISBN=8931004443" ng-show="aladinPreviewLink.length &gt; 0" target="_blank" title="새창">알라딘\xa0<i class="k-icon k-i-s

In [46]:
def compareISBN(text, ISBN):
    regex = re.compile(ISBN)
    matchObj = regex.search(text)
    if matchObj is None :
        return False
    else:
        return True

In [47]:
print(compareISBN(strISBNIncluded, '9791165075064'))
#진짜
print(compareISBN(strISBNIncluded, '9791165075062'))
#가짜.

False
False


In [48]:
loanResultListSelectPath = 'body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div.ikc-content > div.ikc-main > div:nth-child(3) > div > div.ikc-search-detail-main > div > div.ikc-detail-strip > div:nth-child(2) > ng-include > div > table'
loanResultList =bsObject.select(loanResultListSelectPath)[0].find_all('tbody')
len(loanResultList)

2

In [49]:
for item in loanResultList:
    content = item.find('tr').find_all('td')
    print(len(content))
    errorMessage = ''
    RN = content[0].find_all('span')[1].text.strip()
    CN = content[2].find_all('span')[1].text.strip()
    POS = content[1].find_all('span')[1].text.strip()
    STATE_STRING = content[4].find_all('span')[1].text.strip()
    #RDD = content[5].text.strip()
    print(RN)
    print(CN)
    print(POS)
    print(STATE_STRING)
    #print(RDD)
    #print(content[8])
        
    #print('등록번호 : ', item)

6
E0491817
843 H656lㅇ
문학자료실(1층)
대출가능   ~
6
E0491818
843 H656lㅇ c.2
문학자료실(1층)
대출중   ~ 2020.06.19
